In [1]:
!pip install deep_translator
!pip install openpyxl

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer #word lemma class
from deep_translator import GoogleTranslator # import google translator.
from tqdm import tqdm 
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Conv1D,Input, Bidirectional, LSTM, Dense, SpatialDropout1D, Input, Dropout,Embedding,Concatenate, TimeDistributed , BatchNormalization,Flatten,Activation
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pickle
from tensorflow import keras
lemma = WordNetLemmatizer()
Gtrans= GoogleTranslator(source='auto', target='english')
#initialize and download words,stopword and wordnet word corpus.
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Define function for cleaning.
def dataCleaner(text):
    
    #remove x000d from text.
    text=text.replace('_x000D_',' ')
    
    #remove ? from text.
    text=text.replace('?',' ')
    
    #correct acct to account.
    text=text.replace('acct','account')
    
    #correct fw to forward.
    text=text.replace('fw:','forward')
    
    #correct fw to forward.
    text=text.replace(' fw ',' forward ')
    
    #remove (employee# & manager name)    
    #text=text.replace('(employee# & manager name)',' ')
        
    #remove subject:    
    #text=text.replace('subject:',' ')    
    
    #Remove received from: email
   
    
    #Remove hex chars
    lst = re.findall('==pcap 1 hex s==[\w\W]*==pcap 1 hex e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii chars
    lst = re.findall('==pcap 1 ascii s==[\w\W]*==pcap 1 ascii e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove correlation data 
    lst = re.findall('\[correlation_data\](.*\n){2}', text)
    for i in lst:
        text=text.replace(i,' ')
       
    #Remove [correlation_data]
    lst = re.findall('\[correlation_data\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove [no entry] 
    lst = re.findall('\[no entry\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii packet(s):
    text=text.replace('ascii packet(s):',' ')
    
    #Remove hex packet(s)::
    text=text.replace('hex packet(s):',' ')
        
    # Removing white spaces and trimmign the space around words.
    text = re.sub(' +', ' ', text.strip())
    
    #removee tabs
    text=text.replace('\t',' ')
    
    text=text.replace('\r',' ')
    
    #Remove new lines.
    text=text.replace('\n',' ')
    
    # We will use google translate to translate the sentence to english.
    if(len(text)>4000):
        n=3000
        GtransStrings = []
        for index in range(0, len(text), n):
            separated_strings = text[index : index + n]
            GtransStrings.append(Gtrans.translate(separated_strings))
        text=" ".join(GtransStrings)
        
    else:
        if(text!=''):
            text=Gtrans.translate(text)
            
    # Changing to lower case all the words.
    text = text.lower()
    text = text.split()
    
    # Removing english stop words.
    text = [word.strip() for word in text if not word in set(stopwords.words('english'))]
    
    # lemmatizing the words to their base forms
    text = [lemma.lemmatize(word) for word in text]
    
    # Joining back the text.
    text = ' '.join(text)
   
    return text

In [7]:
model1 = keras.models.load_model('./model-0.37.h5')
model2 = pickle.load(open('./lr_model.sav', 'rb'))
max_len = 200
tokenizer = pickle.load(open('./tokenizer.sav', 'rb'))
vectorizer = pickle.load(open('./vectorizer.sav', 'rb'))
smallEncoder = pickle.load(open('./smallEncoder.sav', 'rb'))
FullEncoder = pickle.load(open('./FullEncoder.sav', 'rb'))

NameError: name 'load_model' is not defined

In [107]:
def getGroupAssignment(text):
    text=dataCleaner(text)
    if text.strip() == '' :
        return 'GRP_0'
    X1=pad_sequences(tokenizer.texts_to_sequences([text]), maxlen = max_len)
    model1Pred=FullEncoder.inverse_transform(np.round(model1.predict(X1),0).astype(int))
    
    if model1Pred=='GRP_0':
        return 'GRP_0'
    else:
        X2=vectorizer.transform([text])
        X2=X2.toarray()
        try:
            model2Pred=smallEncoder.inverse_transform(model2.predict(X2))
            model2Pred=model2Pred[0]
        except:
            model2Pred = 'GRP_0'
        return model2Pred
    
    return 'GRP_0'

In [109]:
getGroupAssignment('when undocking pc , screen will not come back')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


'GRP_3'